In [1]:
import pandas as pd
import numpy as np
import os

from tqdm import tqdm
from catboost import CatBoostClassifier
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
train_df = pd.read_csv('../data/train/train_ratings.csv') # 전체 학습 데이터
genre_df = pd.read_csv(os.path.join('../data/', 'train/genres.tsv'), sep='\t')

genre_mulithot

In [3]:
genre_data = genre_df.copy()
genre_dict = {genre:i for i , genre in enumerate(set(genre_df['genre']))}
genre_data['genre'] = genre_data['genre'].map(lambda x : genre_dict[x])


In [4]:
sum_genre = list()
for item in genre_data['item'].unique():
    sum_genre.append([item, genre_data[genre_data['item']==item]['genre'].values])
sum_genre = pd.DataFrame(sum_genre, columns=['item', 'genre'])

In [5]:
mlb = MultiLabelBinarizer()
genre_label = mlb.fit_transform(sum_genre['genre'])
sum_genre = pd.concat([sum_genre['item'], pd.DataFrame(genre_label, columns = genre_dict)], axis=1)
sum_genre

,item,Fantasy,Comedy,Drama,Romance,Horror,Children,Mystery,Thriller,War,Crime,Animation,Musical,Documentary,Action,Sci-Fi,Adventure,Western,Film-Noir
0,318,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,2571,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0
2,2959,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0
3,296,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
4,356,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6802,73106,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6803,109850,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
6804,8605,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6805,3689,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


train, valid split

In [ ]:
items = train_df.groupby("user")["item"].apply(list)

train_set, valid_set = {}, {}
train_unique , valid_unique = [] , []
for uid, item in tqdm(enumerate(items)):
    num_u_valid_set = min(int(len(item)*0.125), 10)
    u_valid_set = np.random.choice(item, size=num_u_valid_set, replace=False)
    
    train_set[uid] = list(set(item) - set(u_valid_set))
    train_unique.extend(train_set[uid])
    valid_set[uid] = u_valid_set
    valid_unique.extend(valid_set[uid])

31360it [00:02, 12598.50it/s]


In [6]:
del train_df['time']
sum_train = pd.merge(train_df, sum_genre, on='item')
sum_train['target'] = 1
sum_train

,user,item,Fantasy,Comedy,Drama,Romance,Horror,Children,Mystery,Thriller,...,Crime,Animation,Musical,Documentary,Action,Sci-Fi,Adventure,Western,Film-Noir,target
0,11,4643,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,1
1,189,4643,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,1
2,294,4643,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,1
3,383,4643,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,1
4,421,4643,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5154466,126426,102880,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,1
5154467,127766,102880,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,1
5154468,130972,102880,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,1
5154469,131347,102880,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,1


negative sampling

In [7]:
item_unique = set(sum_train['item'].unique())
first_row = True
user_neg_dfs = pd.DataFrame()

for uid in tqdm(sum_train['user'].unique()):
    item = set(sum_train[sum_train['user']==uid]['item'])
    #num_negative = min(int(len(item)*0.125), 10)
    num_negative = min(int(len(item)*0.35), 10)

    i_user_neg_item = np.random.choice(list(item_unique - item), num_negative, replace=False)
    
    i_user_neg_df = pd.DataFrame({'user': [uid]*num_negative, 'item': i_user_neg_item, 'target': [0]*num_negative})
    if first_row == True:
        user_neg_dfs = i_user_neg_df
        first_row = False
    else:
        user_neg_dfs = pd.concat([user_neg_dfs, i_user_neg_df], axis = 0, sort=False)

user_neg_dfs = pd.merge(user_neg_dfs, sum_genre , on='item')


100%|██████████| 31360/31360 [05:37<00:00, 93.04it/s] 


In [8]:
user_neg_dfs = user_neg_dfs.rename(columns={"rating" : "target"})
train = pd.concat([sum_train, user_neg_dfs],axis=0)

train = train.sort_values(by=['user'])

In [9]:
train.to_csv("../data/train/test_dataset35%10.csv", index=False)

In [14]:
train_data = pd.read_csv('../data/train/test_dataset.csv')
train_data.head()

,user,item,Crime,Drama,Thriller,Romance,Comedy,Horror,Animation,Children,...,Mystery,Musical,Fantasy,Action,Western,War,Sci-Fi,Film-Noir,Documentary,target
0,11,4643,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
1,11,208,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
2,11,2291,0,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,11,5171,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
4,11,4638,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1


In [15]:
train_labels = train_data['target']
del train_data['target']

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train_data, train_labels, test_size=0.3, random_state=42
)
cat_features = list(range(0, X_train.shape[1]))

y_test = y_test.reset_index()
del y_test['index']

같은 유저, 다른 아이템 - 기존 방식
다른 유저, 같은 아이템 - 지금 방식
positive 는 랜덤으로, negative는 뽑아내기?

In [ ]:
model = CatBoostClassifier(iterations=1000,
                           task_type="GPU",
                           devices='0:1',
                           random_seed=42)
fit_model = model.fit(X_train,
          y_train,
          cat_features,
          verbose=True)

print(fit_model.get_params())

In [25]:
model = CatBoostClassifier(iterations=1000,
                           task_type="GPU",
                           devices='0:1',
                           loss_function="Logloss",
                           random_seed=42)
fit_model = model.fit(X = X_train,
          y = y_train,
          cat_features = cat_features,
          verbose=True)

print(fit_model.get_params())

Learning rate set to 0.02136
0:	learn: 0.6629966	total: 69.9ms	remaining: 1m 9s
1:	learn: 0.6354971	total: 151ms	remaining: 1m 15s
2:	learn: 0.6089293	total: 232ms	remaining: 1m 17s
3:	learn: 0.5848171	total: 303ms	remaining: 1m 15s
4:	learn: 0.5614780	total: 373ms	remaining: 1m 14s
5:	learn: 0.5394395	total: 442ms	remaining: 1m 13s
6:	learn: 0.5199416	total: 512ms	remaining: 1m 12s
7:	learn: 0.5014568	total: 579ms	remaining: 1m 11s
8:	learn: 0.4831503	total: 648ms	remaining: 1m 11s
9:	learn: 0.4670197	total: 717ms	remaining: 1m 10s
10:	learn: 0.4510362	total: 792ms	remaining: 1m 11s
11:	learn: 0.4369733	total: 860ms	remaining: 1m 10s
12:	learn: 0.4226947	total: 937ms	remaining: 1m 11s
13:	learn: 0.4106988	total: 1s	remaining: 1m 10s
14:	learn: 0.3997226	total: 1.07s	remaining: 1m 10s
15:	learn: 0.3882216	total: 1.14s	remaining: 1m 10s
16:	learn: 0.3775727	total: 1.21s	remaining: 1m 9s
17:	learn: 0.3683510	total: 1.28s	remaining: 1m 9s
18:	learn: 0.3593867	total: 1.35s	remaining: 1m 9s

In [17]:
model = CatBoostClassifier(iterations=10,
                           task_type="GPU",
                           devices='0:1',
                           loss_function="Logloss",
                           random_seed=42,
                           custom_loss=["AUC" , "Accuracy", "Recall", "Precision"])
                           
fit_model = model.fit(X = X_train,
          y = y_train,
          cat_features = cat_features,
          eval_set=(X_test, y_test),
          verbose=True)

model.get_evals_result()

Learning rate set to 0.236725
0:	learn: 0.5101025	test: 0.5100908	best: 0.5100908 (0)	total: 42.5ms	remaining: 383ms
1:	learn: 0.4181187	test: 0.4181044	best: 0.4181044 (1)	total: 82.1ms	remaining: 328ms
2:	learn: 0.3211736	test: 0.3211203	best: 0.3211203 (2)	total: 127ms	remaining: 295ms
3:	learn: 0.2758337	test: 0.2757319	best: 0.2757319 (3)	total: 164ms	remaining: 246ms
4:	learn: 0.2441462	test: 0.2441671	best: 0.2441671 (4)	total: 202ms	remaining: 202ms
5:	learn: 0.2275377	test: 0.2275898	best: 0.2275898 (5)	total: 240ms	remaining: 160ms
6:	learn: 0.2177408	test: 0.2178144	best: 0.2178144 (6)	total: 280ms	remaining: 120ms
7:	learn: 0.2117811	test: 0.2118454	best: 0.2118454 (7)	total: 317ms	remaining: 79.3ms
8:	learn: 0.2085667	test: 0.2086222	best: 0.2086222 (8)	total: 355ms	remaining: 39.5ms
9:	learn: 0.2067523	test: 0.2067978	best: 0.2067978 (9)	total: 393ms	remaining: 0us
bestTest = 0.2067978416
bestIteration = 9


{'learn': {'Recall': [0.9999878052261811,
   1.0,
   0.9999894681498838,
   0.9999343145137486,
   0.9961547660917662,
   0.991718917115171,
   0.9871292476960885,
   0.9850273121360449,
   0.983085848713299,
   0.982478050100011],
  'Accuracy': [0.8994361741005035,
   0.8994349276907867,
   0.899440661175484,
   0.8994468932240681,
   0.9092022927956023,
   0.9154448112212771,
   0.9187310950806202,
   0.9197249821888052,
   0.9207657343023421,
   0.9210636262246599],
  'Logloss': [0.510102524677666,
   0.41811873398675115,
   0.32117364057831416,
   0.27583374215572043,
   0.24414619002463403,
   0.22753771822764526,
   0.21774081321254127,
   0.21178108957148933,
   0.20856670797268467,
   0.20675232492804477],
  'Precision': [0.8994448113128762,
   0.8994349276907867,
   0.8994476523667536,
   0.8994928945400149,
   0.911178900124525,
   0.920434001965253,
   0.9272179769173315,
   0.929880625271776,
   0.9324845020777377,
   0.933277273402213]},
 'validation': {'Recall': [0.999988

In [18]:
preds_class = fit_model.predict(X_test)
preds_proba = model.predict_proba(X_test)

print("class = ", preds_class)
print("proba = ", preds_proba)

class =  [1 1 1 ... 1 1 1]
proba =  [[0.02744637 0.97255363]
 [0.00914825 0.99085175]
 [0.01837362 0.98162638]
 ...
 [0.02621856 0.97378144]
 [0.01446918 0.98553082]
 [0.02781899 0.97218101]]


In [25]:
preds_proba[:,1]

array([0.97255363, 0.99085175, 0.98162638, ..., 0.97378144, 0.98553082,
       0.97218101])

In [19]:
correct_result_sum = 0
for i in range(0, len(preds_class)):
    if preds_class[i] == y_test['target'][i]:
        correct_result_sum += 1

print("acc : ", correct_result_sum / len(preds_class))

acc :  0.8994517293848853


In [16]:
correct_result_sum = 0
for i in range(0, len(preds_class)):
    if preds_class[i] == y_test['target'][i]:
        correct_result_sum += 1

print("acc : ", correct_result_sum / len(preds_class))

acc :  0.9263504930131269


In [20]:
def confusion(output, y):
    tp, fp = 0, 0
    fn ,tn = 0, 0
    precision = 0
    recall = 0
    for i in range(0,len(output)):
        if y['target'][i] == 0:
            if output[i] == 0:
                tn += 1
            else:
                fp += 1
        elif y['target'][i] == 1:
            if output[i] == 1:
                tp += 1
            else:
                fn += 1
    precision = tp / (tp+fp)
    recall = tp / (tp + fn)
    return precision , recall

In [21]:
precision , recall = confusion(preds_class, y_test)

In [22]:
print("precision : ", precision)
print("recall : ", recall)

precision :  0.8996358913272072
recall :  0.9997445627431504


In [2]:
model = CatBoostClassifier()
model.load_model("./output/catboost_iter_5.bin")

In [12]:
model.get_params()

{'devices': '0:1',
 'task_type': 'GPU',
 'random_seed': 42,
 'custom_metric': ['AUC', 'Accuracy', 'Recall', 'Precision'],
 'loss_function': 'Logloss',
 'iterations': 5,
 'verbose': 1}

In [32]:
submission = pd.read_csv('../data/train/submission-RecVAE-top50.csv')
users_set = set(submission['user'].unique())
items_set = set(submission['item'].unique())

submission_cat = pd.DataFrame(columns=['user', 'item'])

In [21]:
submission['user'].value_counts()

11        50
91671     50
91660     50
91655     50
91643     50
          ..
45730     50
45729     50
45723     50
45722     50
138493    50
Name: user, Length: 31360, dtype: int64

In [ ]:
for user in tqdm(user_set):
    preds_items = submission[submission['user']==user]['item'].values
    
    x = pd.DataFrame({'user' :[user] * len(preds_items) , 'item' : preds_items, 'y' : 0 * len(preds_items)})

    x = pd.merge(x, sum_genre, on='item')

    preds_proba = model.predict_proba(x)
    x['y'] = preds_proba[:,1]
    output = x[['user','item','y']]
    output = output.sort_values(by='y', ascending=False)[:10]
    output.drop(columns='y', inplace=True)

    submission_cat = pd.concat([submission_cat, output])

In [26]:
submission_genre = pd.merge(submission, sum_genre, on='item')
submission_genre['y'] = 0
submission_genre

,user,item,Drama,Film-Noir,Horror,Sci-Fi,Adventure,Western,Comedy,Children,...,Musical,Mystery,Documentary,Thriller,War,Action,Crime,Romance,Animation,y
0,11,2161,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,190,2161,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,294,2161,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,462,2161,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,789,2161,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567995,137525,3480,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1567996,137643,71460,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1567997,138325,5974,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1567998,138325,25945,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [27]:
preds_proba = model.predict_proba(submission_genre)
submission_genre['y'] = preds_proba[:,1]
submission_genre = submission_genre[['user','item','y']]

In [33]:
remove_sub = pd.DataFrame(columns=['user', 'item'])
for user in tqdm(users_set):
    output = submission_genre[submission_genre['user']==user].sort_values(by='y', ascending=False)[:10]
    output.drop(columns='y' , inplace=True)
    remove_sub = pd.concat([remove_sub, output])

100%|██████████| 31360/31360 [04:12<00:00, 124.16it/s]


In [36]:
remove_sub[remove_sub['user']==11]

,user,item
83206,11,2987
86822,11,1127
8516,11,34405
33560,11,40815
20457,11,4886
85208,11,2
80250,11,33004
86226,11,4370
18894,11,2005
35877,11,2115


In [43]:
remove_sub.sort_values(by='user', ascending=True, inplace=True)
remove_sub

,user,item
83206,11,2987
35877,11,2115
18894,11,2005
86226,11,4370
80250,11,33004
...,...,...
651385,138493,44191
150691,138493,2011
856411,138493,924
175952,138493,2628
